In [1]:
import numpy as np
import os
import os.path
import nibabel as nib
import pandas as pd
from src.utils.data import writeData, getDataPandas, getMeta, writeBlock, getDataTagPandas
from src.process.ml import readVoxels, imgPCA
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler

In [10]:
voxels = readVoxels()
data = getDataPandas()
scores = data['SCORE']
demos = data[['AGE_GAP']]

scaler = MinMaxScaler()
demos = scaler.fit_transform(demos)

#labels = np.append(voxels, demos, axis=1)
labels = voxels

train_labels = labels[:200]
val_labels = labels[201:]
train_scores = scores[:200]
val_scores = scores[201:]

In [11]:
print(train_labels.shape)
pca, train_features = imgPCA(train_labels, 0.999)
print(train_features.shape)
train_labels_demo = np.append(train_features, demos[:200], axis=1)
#train_labels_demo = train_features

(200, 122)
(200, 10)


In [6]:
model = linear_model.LinearRegression()
model.fit(train_labels_demo, train_scores)

LinearRegression()

In [7]:
print(model.coef_)
print(model.intercept_)

[-1.05704839e-01  9.45956052e-02  1.33012708e-02  1.67186092e-01
  7.27202113e-01  3.84440156e-01 -3.34158514e-01 -7.56959615e-01
  9.35881888e-01  5.04468610e+00  1.56674930e-02  5.87464300e-04]
0.33589911675311757


In [8]:
train_features = pca.transform(train_labels)
train_labels_demo = np.append(train_features, demos[:200], axis=1)
#train_labels_demo = train_features
scores_pred = np.dot(train_labels_demo, model.coef_) + model.intercept_
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
print(MSE(train_scores, scores_pred))
print(MSE(train_scores, scores_pred)**0.5)
print(MAE(train_scores, scores_pred))
print(r2_score(train_scores, scores_pred))

0.033011661325411515
0.18169111515264447
0.14553263145118941
0.1684451731555664


In [9]:
val_features = pca.transform(val_labels)
val_labels_demo = np.append(val_features, demos[201:], axis=1)
#val_labels_demo = val_features
scores_pred = np.dot(val_labels_demo, model.coef_) + model.intercept_
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score
print(MSE(val_scores, scores_pred))
print(MSE(val_scores, scores_pred)**0.5)
print(MAE(val_scores, scores_pred))
print(r2_score(val_scores, scores_pred))

0.058391598429372915
0.24164353587334572
0.19236427023042887
-0.005213320976943159
